In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from backend.models import (
    SimulationParams,
    UniverseParams,
    FinancingParams,
    StrategyParams,
    OptionsParams,
)
from backend.simulator import run_simulation


In [2]:
params = SimulationParams(
    universe=UniverseParams(
        n_assets=50,
        mean_return=6.0,
        volatility=20.0,
        ic=0.05,
        factor_autocorr=0.8,
    ),
    financing=FinancingParams(
        cash_rate=3.0,
        margin_rate=5.0,
        borrow_fee=1.0,
    ),
    strategy=StrategyParams(
        strategy_length=60,
        risk_aversion=4.0,
        long_weight=130.0,
        short_weight=30.0,
        turnover_limit=25.0,
        max_weight=10.0,
        lookback=36,
        transaction_cost_bps=5.0,
    ),
    options=OptionsParams(
        enabled=True,
        call_otm_pct=0.0,
        put_otm_pct=0.0,
        call_alpha_barrier=0.0,
        put_alpha_barrier=0.0,
        contract_fee=0.65,
        spread_bps=100.0,
    ),
)


In [3]:
np.random.seed(42)
sim = run_simulation(params)

base = sim.base
overlay = sim.with_options
benchmark = sim.benchmark

lookback = params.strategy.lookback


In [4]:
n_base = len(base.portfolio_returns)
n_overlay = len(overlay.portfolio_returns)
n_bench = len(benchmark.returns)
n = min(n_base, n_overlay, n_bench)

months = np.arange(1, n + 1)

df_base = pd.DataFrame(
    {
        "month": months,
        "portfolio_return": base.portfolio_returns[:n],
        "benchmark_return": benchmark.returns[:n],
        "turnover": base.turnovers[:n],
        "financing_cost": base.financing_costs[:n],
    }
)

df_overlay = pd.DataFrame(
    {
        "month": months,
        "portfolio_return": overlay.portfolio_returns[:n],
        "benchmark_return": benchmark.returns[:n],
        "turnover": overlay.turnovers[:n],
        "financing_cost": overlay.financing_costs[:n],
        "options_income": overlay.options_income[:n],
    }
)


In [5]:
for df in (df_base, df_overlay):
    df["portfolio_cumulative"] = (1 + df["portfolio_return"] / 100).cumprod() - 1
    df["benchmark_cumulative"] = (1 + df["benchmark_return"] / 100).cumprod() - 1

start_base = pd.DataFrame(
    {
        "month": [0],
        "portfolio_return": [0.0],
        "benchmark_return": [0.0],
        "turnover": [0.0],
        "financing_cost": [0.0],
        "portfolio_cumulative": [0.0],
        "benchmark_cumulative": [0.0],
    }
)

start_overlay = pd.DataFrame(
    {
        "month": [0],
        "portfolio_return": [0.0],
        "benchmark_return": [0.0],
        "turnover": [0.0],
        "financing_cost": [0.0],
        "options_income": [0.0],
        "portfolio_cumulative": [0.0],
        "benchmark_cumulative": [0.0],
    }
)

df_base = pd.concat([start_base, df_base], ignore_index=True)
df_overlay = pd.concat([start_overlay, df_overlay], ignore_index=True)

df_base["month"] = range(len(df_base))
df_overlay["month"] = range(len(df_overlay))


In [6]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_base["month"],
        y=df_base["portfolio_cumulative"] * 100,
        mode="lines",
        name="Strategy w/o Options",
        line=dict(color="blue"),
    )
)
fig.add_trace(
    go.Scatter(
        x=df_overlay["month"],
        y=df_overlay["portfolio_cumulative"] * 100,
        mode="lines",
        name="Strategy w/ Options",
        line=dict(color="green"),
    )
)
fig.add_trace(
    go.Scatter(
        x=df_base["month"],
        y=df_base["benchmark_cumulative"] * 100,
        mode="lines",
        name="Benchmark",
        line=dict(color="gray", dash="dash"),
    )
)

fig.update_layout(
    title="Cumulative Returns: With vs Without Options Overlay",
    xaxis_title="Month",
    yaxis_title="Return (%)",
    hovermode="x unified",
    yaxis=dict(hoverformat=".2f"),
)
fig.show()


In [7]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Turnover (left axis)
fig.add_trace(
    go.Scatter(
        x=df_base["month"][1:],
        y=df_base["turnover"][1:],
        name="Turnover (w/o Options)",
        line=dict(color="blue"),
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=df_overlay["month"][1:],
        y=df_overlay["turnover"][1:],
        name="Turnover (w/ Options)",
        line=dict(color="green"),
    ),
    secondary_y=False,
)

# Options income (right axis)
fig.add_trace(
    go.Scatter(
        x=df_overlay["month"][1:],
        y=df_overlay["options_income"][1:],
        name="Options Income",
        line=dict(color="orange", width=2),
    ),
    secondary_y=True,
)

fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Turnover (%)", secondary_y=False)
fig.update_yaxes(title_text="Options Income (%)", secondary_y=True)

fig.update_layout(
    title="Turnover vs Options Income",
    hovermode="x unified",
    yaxis=dict(hoverformat=".2f"),
    yaxis2=dict(hoverformat=".2f"),
)
fig.show()

# Simple correlation between turnover and options income (overlay leg)
corr = np.corrcoef(
    df_overlay["turnover"][1:],
    df_overlay["options_income"][1:],
)[0, 1]
print(f"\nCorrelation between turnover and options income (overlay): {corr:.3f}")



Correlation between turnover and options income (overlay): 0.052
